In [12]:
from bs4 import BeautifulSoup
import requests
import re
import sqlite3
from pathlib import Path
import time

In [2]:
database_path = Path('/database') 

In [3]:
conn = sqlite3.connect('database/amazon_data.sqlite')
c = conn.cursor()

In [29]:
c.execute('''
    CREATE TABLE IF NOT EXISTS products (
        price_per_tb REAL,
        price REAL,
        capacity TEXT,
        technology TEXT,
        name TEXT,
        url TEXT
    )
''')

# Commit the changes and close the connection
conn.commit()

In [5]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
            , 'Accept-Language': 'en-US, en;q=0.5'})

url = 'https://amzn.to/3RlUHdO'
response = requests.get(url, headers=HEADERS)


In [38]:
soup = BeautifulSoup(response.content, "html")

In [39]:
product_name = soup.find("span", attrs={"id": 'productTitle'}).string.strip().replace(',', '')


In [40]:
product_price = find_price = soup.find("input", attrs={'id': 'attach-base-product-price'}).get('value').strip().replace(',', '')
price = float(product_price)

In [51]:
table2= soup.find('table', {'class': 'a-normal a-spacing-micro'})
#flash_mem_size = table2.find('th', string=' Flash Memory Size ').find_next('td')

In [43]:
rows = table2.find_all('tr')
table_data = []
for row in rows:
    # Extract text from each cell in the row
    cells = row.find_all(['td'])
    row_data = [cell.find('span').get_text(strip=True) for cell in cells]
    table_data.append(row_data)
    print(row_data)

['Digital Storage Capacity', '4 TB']
['Hard Disk Interface', 'NVMe']
['Connectivity Technology', 'NVMe']
['Brand', 'Crucial']
['Special Feature', 'Portable']
['Hard Disk Form Factor', '3.5 Inches']
['Hard Disk Description', 'Solid State Drive']
['Compatible Devices', 'Desktop']
['Installation Type', 'Internal Hard Drive']
['Color', 'Black']


In [47]:
print(table_data)
print(table_data[0][1])

[['Digital Storage Capacity', '4 TB'], ['Hard Disk Interface', 'NVMe'], ['Connectivity Technology', 'NVMe'], ['Brand', 'Crucial'], ['Special Feature', 'Portable'], ['Hard Disk Form Factor', '3.5 Inches'], ['Hard Disk Description', 'Solid State Drive'], ['Compatible Devices', 'Desktop'], ['Installation Type', 'Internal Hard Drive'], ['Color', 'Black']]
4 TB


In [45]:
size = int(re.search(r'\d+', table_data[0][1]).group())
size

4

In [46]:
if table_data[0][1].split(' ')[1] == 'TB':
    size = int(re.search(r'\d+', table_data[0][1]).group())
    price_per_terabyte = price/size
elif table_data[0][1].split(' ')[1] == 'GB':
    size = int(re.search(r'\d+', table_data[0][1]).group())
    size = size / 1000
    price_per_terabyte = price/size
else: 
    price_per_terabyte = 'NA'

In [10]:
size = int(re.search(r'\d+', capacity).group())
price_per_terabyte = price/size

In [11]:
print(product_name)
print(product_price)
print(capacity)
print(technology)
print(price_per_terabyte)

Crucial T700 4TB Gen5 NVMe M.2 SSD - Up to 12400 MB/s - DirectStorage Enabled - CT4000T700SSD3 - Gaming Photography Video Editing & Design - Internal Solid State Drive
449.99
4 TB
NVMe
112.4975


In [31]:
rows = [(price_per_terabyte, price, capacity, technology, product_name, url)]
c.executemany('insert into products values (?,?,?,?,?,?)', rows)
conn.commit()


In [ ]:
'https://amzn.to/41vdlop'
,'https://amzn.to/3tyiGi4'
,'https://amzn.to/4868g8l'
,'https://amzn.to/3RAYiF0'
,'https://amzn.to/3NEgcWe'
,'https://amzn.to/3NCHmwP'
,'https://amzn.to/48uG4w2'

In [24]:
urls = [
    'https://amzn.to/41vdlop','https://amzn.to/3tyiGi4','https://amzn.to/4868g8l','https://amzn.to/3RAYiF0','https://amzn.to/3NEgcWe'
,'https://amzn.to/3NCHmwP','https://amzn.to/48uG4w2','https://amzn.to/3NC0yLe','https://amzn.to/48t7m5S','https://amzn.to/487654F'
,'https://amzn.to/3RTWyZ8','https://amzn.to/3trvSFC','https://amzn.to/48uhaMK','https://amzn.to/486bB7H','https://amzn.to/4ayTskk'
,'https://amzn.to/4axrPZa','https://amzn.to/3vgOdFI','https://amzn.to/4attBdH','https://amzn.to/3vca29F','https://amzn.to/41uc0yh'
,'https://amzn.to/41F19la','https://amzn.to/4743LKk','https://amzn.to/3veaG6E','https://amzn.to/41wgx2W','https://amzn.to/3RwmBnO'
,'https://amzn.to/48oLPLx','https://amzn.to/3GRXR4f','https://amzn.to/3NAlYIo','https://amzn.to/4azreG7','https://amzn.to/3GReXzi'
,'https://amzn.to/3GReM76','https://amzn.to/3NEgGf0','https://amzn.to/3GYkcgs','https://amzn.to/3vjQKio','https://amzn.to/3TzPAtx'
,'https://amzn.to/3GYkfc8'
]

In [48]:
def scrape_amzn(url):
    conn = sqlite3.connect('database/amazon_data.sqlite')
    c = conn.cursor()
    HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
            , 'Accept-Language': 'en-US, en;q=0.5'})

    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "lxml")

    try:
        product_name = soup.find("span", attrs={"id": 'productTitle'}).string.strip().replace(',', '')
    except:
        product_name = 'Name not found'

    try:
        product_price = find_price = soup.find("input", attrs={'id': 'attach-base-product-price'}).get('value').strip().replace(',', '')
        price = float(product_price)
    except:
        price = 'NA'

    try:
        #extract the table
        table= soup.find('table', {'class': 'a-normal a-spacing-micro'})
        
        #extract the rows
        rows = table.find_all('tr')
        
        #create an empty list to store the row data
        table_data = []
        
        #loop through the rows and append the data to the empty list
        for row in rows:
            cells = row.find_all(['td'])
            row_data = [cell.find('span').get_text(strip=True) for cell in cells]
            table_data.append(row_data)
    except:
        print(url + ' No product details found')
   
    #Extract storage capacity
    try:
        capacity = table_data[0][1]
    except:
        capacity = 'NA'

    #extract Technology
    try:
        technology = table_data[1][1]
    except:
        technology = 'NA'
        
    #compute price_per_terabyte
    if capacity != 'NA' and price != 'NA':
        if table_data[0][1].split(' ')[1] == 'TB':
            size = int(re.search(r'\d+', table_data[0][1]).group())
            price_per_terabyte = price/size
        elif table_data[0][1].split(' ')[1] == 'GB':
            size = int(re.search(r'\d+', table_data[0][1]).group())
            size = size / 1000
            price_per_terabyte = price/size
        else: 
            price_per_terabyte = 'NA'
    else:
        price_per_terabyte = 'NA'
        
    rows = [(price_per_terabyte, price, capacity, technology, product_name, url)]
    c.executemany('insert into products values (?,?,?,?,?,?)', rows)
    conn.commit()
    time.sleep(5)

In [49]:
urls = [
    'https://amzn.to/41vdlop','https://amzn.to/3tyiGi4','https://amzn.to/4868g8l','https://amzn.to/3RAYiF0','https://amzn.to/3NEgcWe']

In [50]:

for url in urls:
    scrape_amzn(url)